In [76]:
import pandas as pd
import numpy as np

In [77]:
df_meta = pd.read_csv('../datasets/raw/HT_Sensor_metadata.dat', delimiter='\t+')
df_meta.shape

(100, 5)

In [83]:
df_meta.head()

,id,date,class,t0,dt
0,0,07-04-15,banana,13.49,1.64
1,1,07-05-15,wine,19.61,0.54
2,2,07-06-15,wine,19.99,0.66
3,3,07-09-15,banana,6.49,0.72
4,4,07-09-15,wine,20.07,0.53


In [87]:
df_db = pd.read_csv('../datasets/raw/HT_Sensor_dataset.dat', delimiter='\s+')
df_db.shape

(928991, 12)

# Checking shubham article

In [88]:
df_db.set_index('id', inplace=True)
df_db = df_db.join(df_meta, how='inner')

In [89]:
df_db.set_index(np.arange(df_db.shape[0]), inplace=True)
df_db['time'] += df_db['t0']
df_db.drop(['t0'], axis=1, inplace=True)
df_db.head()

,time,R1,R2,R3,R4,R5,R6,R7,R8,Temp.,Humidity,id,date,class,dt
0,12.490250,12.8621,10.3683,10.4383,11.6699,13.4931,13.3423,8.04169,8.73901,26.2257,59.0528,0,07-04-15,banana,1.64
1,12.490528,12.8617,10.3682,10.4375,11.6697,13.4927,13.3412,8.04133,8.73908,26.2308,59.0299,0,07-04-15,banana,1.64
2,12.490806,12.8607,10.3686,10.4370,11.6696,13.4924,13.3405,8.04101,8.73915,26.2365,59.0093,0,07-04-15,banana,1.64
3,12.491084,12.8602,10.3686,10.4370,11.6697,13.4921,13.3398,8.04086,8.73936,26.2416,58.9905,0,07-04-15,banana,1.64
4,12.491373,12.8595,10.3688,10.4374,11.6699,13.4919,13.3390,8.04087,8.73986,26.2462,58.9736,0,07-04-15,banana,1.64


In [91]:
from sklearn.model_selection import train_test_split

In [93]:
features = ['R1', 'R2', 'R3', 'R4', 'R5', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity']
xtrain,xtest,ytrain,ytest = train_test_split(df_db[features].values, df_db['class'].values, train_size=0.7)

In [94]:
from sklearn.linear_model import LogisticRegressionCV

clf_lr = LogisticRegressionCV()
###
# Default: k-fold: 5-fold cross validation
#          Cs: A grid of Cs values are chosen in a logarithmic scale between 1e-4 and 1e4
#          penalty: l2
#          solver: lbfgs
#          tol: 1e-4
#          max_iter: 100
###

In [96]:
clf_lr.fit(xtrain, ytrain)

LogisticRegressionCV(Cs=10, class_weight=None, cv='warn', dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='warn', n_jobs=None, penalty='l2',
           random_state=None, refit=True, scoring=None, solver='lbfgs',
           tol=0.0001, verbose=0)

In [100]:
clf_lr.score(xtest, ytest)

0.5759280655045964

In [103]:
from sklearn.svm import SVC

In [104]:
C = 1e-2
clf_svm = SVC(C=C)

In [105]:
clf_svm.fit(xtrain, ytrain)

SVC(C=0.01, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [108]:
clf_svm.score(xtest, ytest)

0.9369999067090542

In [109]:
from datetime import datetime
now = datetime.now()
print(now)

2020-12-06 01:28:32.381654


In [110]:
import pickle
with open('svc_test_model.pkl', 'wb') as f:
    pickle.dump(clf_svm, f)

In [111]:
with open('svc_test_model.pkl', 'rb') as f:
    clf_svm = pickle.load(f)